In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

# import win32com.client as win32
# from pathlib import Path
import xlsxwriter
from itertools import chain
from datetime import datetime, timedelta
import pandas as pd
import glob
import os

import sys
sys.path.append('../utils')
from SharePoint import *
from hlpr import *
from static import *
from format_qa import *

import warnings
warnings.filterwarnings('ignore')

# win32c = win32.constants

In [2]:
media = 'search'

SHAREPOINT_DATA_PATH = media_args[media]['sharepoint']['data']
SHAREPOINT_MAPPING_PATH = media_args[media]['sharepoint']['mapping']
SHAREPOINT_QA_PATH = media_args[media]['sharepoint']['qa']
SHAREPOINT_FLAT_PATH = media_args[media]['sharepoint']['flat']
REDSHIFT_METRICS_QUERY = media_args[media]['redshift']['all_metrics']
REDSHIFT_METRICS_FILENAME = media_args[media]['redshift']['metric_filename']
REDSHIFT_METRICS_SHEETNAME = media_args[media]['redshift']['metric_sheetname']
REDSHIFT_QA_QUERY = media_args[media]['redshift']['qa_qry']

**Connect to SharePoint**

In [3]:
app = SharePoint('AmericanExpressUSGABM')

# download data
app.list_contents(SHAREPOINT_DATA_PATH)
app.download_files(DATA_PATH)
app.archive_files()

# download floodlight mapping
app.list_contents(SHAREPOINT_MAPPING_PATH)
app.download_files(ASSETS_PATH)

connected to: https://interpublic.sharepoint.com/sites/AmericanExpressUSGABM

listing files from: Measurement%20%20Analytics%20Folder/GABM/Global%20Analytics/DigitalQA/Search/01Sandbox/SA360_Files/Gmail/National

downloading data to: C:\Users\carmelo.urena\Documents\Main\OneDrive_BAK\Amex\DigitalQA\SharePoint\sharepoint_api\data

archiving data in SharePoint

listing files from: Measurement%20%20Analytics%20Folder/GABM/Global%20Analytics/DigitalQA/Search/02Mapping

downloading data to: C:\Users\carmelo.urena\Documents\Main\OneDrive_BAK\Amex\DigitalQA\SharePoint\sharepoint_api\assets



**Import Floodlight Metrics**

In [4]:
# import metrics
pth = glob.glob(os.path.join(ASSETS_PATH, f"*{REDSHIFT_METRICS_FILENAME}*"))[0]
df_metrics = pd.read_excel(pth, sheet_name='FloodLight')

**Import SA360 Data** <br>
* Import data that was downloaded from the BAE SharePoint site <br>
* Get list of floodlights by their custom floodlight names, as they appear in SA360

In [5]:
# import sa360 data
list_of_floodlights = []
list_of_dfs = []

files = glob.glob(os.path.join(DATA_PATH, '*.xlsx'))
for f in files:
    df = pd.read_excel(f)
    
    # column check and add floodlights
    if list(df.columns[:17]) == sa360_col_check:
        list_of_floodlights.append(list(df.columns[17:]))
    else:
        print(f'\ncolumn name mismatch: {f}\n')
        
    # transpose sa360 data
    id_vars = list(df.columns[:11])
    df_trans = df.melt(id_vars = id_vars)
    list_of_dfs.append(df_trans)

# combine sa360 data
df_ui = pd.concat(list_of_dfs)
df_ui = df_ui[-df_ui['variable'].isin(['CTR', 'Avg CPC', 'Avg pos'])].reset_index(drop=True)

**Get Parameters for DB** <br>
* advertiser <br>
* start date <br>
* end date <br>
* metrics <br>
     * floodlights_columns: custom SA360 floodlight names <br>
     * floodlights_names: real floodlight names from the database

In [6]:
# get the list of advertisers/accounts for database pull
db_advertisers = "','".join(list(df_ui['Account'].unique()))

# get dates
db_start_date = datetime.strftime(df_ui['From'].min(), '%Y-%m-%d')
db_end_date = datetime.strftime(df_ui['From'].max() + timedelta(days=6), '%Y-%m-%d')

In [7]:
# adjust floodlights
floodlights_columns = list(chain(*list_of_floodlights)) # unpack list
floodlights_columns = list(dict.fromkeys(floodlights_columns,None).keys()) # remove dupes

# filter metrics
floodlights_names = df_metrics[df_metrics['sa360_col_name'].isin(floodlights_columns)]
db_metrics = ['clicks', 'impr', 'cost']

if len(floodlights_names['sa360_col_name'].unique()) == len(floodlights_columns):
    # metrics for database
#     db_metrics = list(floodlights_names['metric']) + db_metrics
    db_metrics = "','".join(list(floodlights_names['metric']) + db_metrics)
else:
    db_metrics = "','".join(db_metrics)


**Query Database**

In [8]:
# get data from redshift
qry_txt = get_qry_text(REDSHIFT_QA_QUERY)
qry_txt = qry_txt.replace('load_metrics', db_metrics) \
                 .replace('load_advertisers', db_advertisers) \
                 .replace('load_start_date', db_start_date) \
                 .replace('load_end_date', db_end_date)

# print(f"Query Parameters\n{'-'*50}")
# print(f"advertisers: {db_advertisers}\n\nmetrics: {db_metrics}\n\nstart_date: {db_start_date}\nend_date: {db_end_date}")

time.sleep(1)
df_qry_raw = run_qry(qry_txt)

In [9]:
# map floodlights
df_qry = df_qry_raw.copy()
df_qry = df_qry.merge(floodlights_names, how='left', on=['advertiser', 'metric'])
df_qry['sa360_col_name'] = df_qry.apply(lambda x: x['metric'].title() if x['metric'] in ['impr', 'cost', 'clicks'] else x['sa360_col_name'], axis=1)

In [10]:
# null results
df_qry_na = df_qry[df_qry['sa360_col_name'].isna()]
df_qry_na_metrics = df_qry_na[['advertiser', 'metric']].drop_duplicates().reset_index(drop=True)

if len(df_qry_na_metrics.merge(floodlights_names, how='inner')) > 0:
    print('advertiser and metric combination is not in df_qry, check sa360_metrics.xlsx')
    print(df_qry_na_metrics)

In [11]:
df_qry = df_qry[-df_qry['sa360_col_name'].isna()] 

**Combine DB and UI Data**

In [12]:
# align ui column names
ui_col_rename = {
    'From': 'week',
    'Account': 'advertiser',
    'Campaign': 'campaign_name',
    'variable': 'sa360_col_name',
    'value': 'value'
}

df_ui_fltr = df_ui[ui_col_rename.keys()]
df_ui_fltr.rename(columns=ui_col_rename, inplace=True)

In [13]:
df_qry_fltr = df_qry[ui_col_rename.values()]

In [14]:
# add source column
df_qry_fltr['source'], df_ui_fltr['source'] = 'redshift', 'sa360' 

In [15]:
df_qa = pd.concat([df_qry_fltr, df_ui_fltr])

In [16]:
df_qa['advertiser'] = df_qa['advertiser'].str.replace('Amex - Shop small', 'Amex - Shop Small')

In [17]:
df_qa['week'] = pd.to_datetime(df_qa['week'])
df_qa['week'] = df_qa['week'].apply(lambda x: x - timedelta(days=x.weekday()))

**QA Pivots**

In [18]:
pvts_dict = dict()

In [19]:
db_params_dict = {'last_updated':datetime.now(), 'advertisers': db_advertisers, 'start_date': db_start_date, 'end_date': db_end_date, 'sql_qry': qry_txt}
details = pd.DataFrame.from_dict(db_params_dict, orient='index').reset_index()
details.columns = ['variable', 'value']

pvts_dict['details'] = details

In [20]:
# create qa views
#====================
# raw data view
df_v0 = df_qa.copy()

df_v0 = df_v0.pivot_table(index=['advertiser', 'campaign_name', 'week', 'sa360_col_name'], columns=['source'], values='value', aggfunc='sum').reset_index().fillna(0)
df_v0 = df_v0.melt(id_vars=['advertiser', 'campaign_name', 'week', 'sa360_col_name'])
df_v0 = df_v0.sort_values(by=['advertiser', 'campaign_name', 'week', 'sa360_col_name']).reset_index(drop=True)
df_v0.rename(columns={'sa360_col_name':'metric'}, inplace=True)

# redshift v sa360
df_v1 = df_qa.copy()

df_v1 = df_v1.pivot_table(index=['advertiser', 'campaign_name', 'week', 'sa360_col_name'], columns=['source'], values='value', aggfunc='sum').reset_index()
df_v1['%_diff'] = abs((df_v1['redshift']/df_v1['sa360'])-1)
df_v1.rename(columns={'sa360_col_name':'metric'}, inplace=True)

In [21]:
view_args = {
    'advertiser': {'index':['advertiser'],
                   'dim_cutoff': 1},
        
    'campaign': {'index':['advertiser', 'campaign_name'], 
                 'dim_cutoff': 2},
    
    'week': {'index':['advertiser', 'campaign_name', 'week'],
             'dim_cutoff': 3}
}

In [22]:
# pvts_dict = dict()

for k in view_args.keys():
    df_pvt = df_qa_week = df_qa.pivot_table(index=view_args[k]['index'], columns=['sa360_col_name', 'source'], values='value', aggfunc='sum').reset_index()
    
    list_of_metric_diff = []
    metrics = df_pvt.columns.levels[0][:list(df_pvt.columns.levels[0]).index(view_args[k]['index'][-1])]
    for m in metrics:
        df_temp = df_pvt.copy()
        df_temp = df_temp[m]
        df_temp.columns = [f"{m}_{c}" for c in df_temp.columns]
        
        # fill na based on condition 
        col0 = df_temp.columns[0]
        col1 = df_temp.columns[1]
        df_temp[col0] = df_temp.apply(lambda x: 0 if np.isnan(x[col0]) and not(np.isnan(x[col1])) else x[col0], axis=1)
        df_temp[col1] = df_temp.apply(lambda x: 0 if np.isnan(x[col1]) and not(np.isnan(x[col0])) else x[col1], axis=1)
        
        df_temp[f"{m}_%_diff"] = (df_temp.iloc[:,1]/df_temp.iloc[:,0])-1
        df_temp[f"{m}_%_diff"] = df_temp[f"{m}_%_diff"].apply(lambda x: 1 if x == float('inf') else x)
        
        list_of_metric_diff.append(df_temp)
        
#         df_temp[f"{m}_%_diff"] = df_temp[f"{m}_%_diff"].apply(lambda x: 1 if x == float('inf') else x)

    metric_diffs = pd.concat(list_of_metric_diff, axis=1)
    
    df_pvt_qa = pd.concat([df_pvt.iloc[:, :view_args[k]['dim_cutoff']], metric_diffs], axis=1)
    df_pvt_qa.columns = [c[0] if type(c) is tuple else c for c in df_pvt_qa.columns]
    
    pvts_dict[k] = df_pvt_qa
    

In [23]:
pvts_dict['raw_data'], pvts_dict['redshift_v_sa360'] = df_v0, df_v1

In [61]:
# clear out no value campaigns
campaign_qa = pvts_dict['campaign'].copy()
remove_campaigns = list(campaign_qa[campaign_qa.iloc[:, 2:].sum(axis=1) == 0.0]['campaign_name'].unique())

# clean dictionary
keys = list(pvts_dict.keys())
for k in keys[2:]:
    pvts_dict[k] = pvts_dict[k][-(pvts_dict[k]['campaign_name'].isin(remove_campaigns))]
    pvts_dict[k].reset_index(drop=True)

**Export**

In [120]:
qa_filename = f"QA_Search_{app.dt}.xlsx"
writer = pd.ExcelWriter(os.path.join(OUTPUTS_PATH, qa_filename), engine='xlsxwriter')

for k in pvts_dict.keys():
    pvts_dict[k].to_excel(writer, sheet_name=f"{k}", index=False)
        
writer.save()

**Format**

In [25]:
format_qa(media, os.path.join(OUTPUTS_PATH, qa_filename))

**Upload to SharePoint**

In [29]:
local_outputs_path = max(glob.glob(os.path.join(OUTPUTS_PATH, 'QA*')), key=os.path.getmtime)
app.upload_files(SHAREPOINT_QA_PATH, local_outputs_path)

# Function

In [ ]:
def qa():
# initiate SharePoint
# ========================================================================================================================
    app = SharePoint('AmericanExpressUSGABM')

    # download data
    app.list_contents(SHAREPOINT_DATA_PATH)
    app.download_files(DATA_PATH)
    app.archive_files()

    # download floodlight mapping
    app.list_contents(SHAREPOINT_MAPPING_PATH)
    app.download_files(ASSETS_PATH)

# qa
# ========================================================================================================================
    if media == 'search':
        qa_resutls = search_qa() # return dictionary

# save qa file
# ========================================================================================================================
    qa_filename = f"QA_{media.title()}_{app.dt}.xlsx"
    writer = pd.ExcelWriter(os.path.join(OUTPUTS_PATH, qa_filename), engine='xlsxwriter')

    for k in qa_resutls.keys():
        qa_resutls[k].to_excel(writer, sheet_name=f"{k}", index=False)
    writer.save()

# format qa file
# ========================================================================================================================
    format_qa(media, os.path.join(OUTPUTS_PATH, qa_filename))

# upload to SharePoint
# add script to run multiple times on error (max 3)
# ========================================================================================================================
    local_outputs_path = max(glob.glob(os.path.join(OUTPUTS_PATH, 'QA*')), key=os.path.getmtime)
    app.upload_files(SHAREPOINT_QA_PATH, local_outputs_path)
    

In [32]:
def search_qa(redshift_metrics_filename, redshift_qa_query):
# import metrics
# ========================================================================================================================
    pth = glob.glob(os.path.join(ASSETS_PATH, f"*{redshift_metrics_filename}*"))[0]
    df_metrics = pd.read_excel(pth, sheet_name='FloodLight')

# import sa360 data
# add to error log for metric mismatches
# ========================================================================================================================
    list_of_floodlights = []
    list_of_dfs = []

    files = glob.glob(os.path.join(DATA_PATH, '*.xlsx'))
    for f in files:
        df = pd.read_excel(f)

        # column check and add floodlights
        if list(df.columns[:17]) == sa360_col_check:
            list_of_floodlights.append(list(df.columns[17:]))
        else:
            print(f'\ncolumn name mismatch: {f}\n')

        # transpose sa360 data
        id_vars = list(df.columns[:11])
        df_trans = df.melt(id_vars = id_vars)
        list_of_dfs.append(df_trans)

# combine sa360 data
# ========================================================================================================================
    df_ui = pd.concat(list_of_dfs)
    df_ui = df_ui[-df_ui['variable'].isin(['CTR', 'Avg CPC', 'Avg pos'])].reset_index(drop=True)

# create database aruments
# add to error log if floodlight mismatch
# ========================================================================================================================
    db_advertisers = "','".join(list(df_ui['Account'].unique()))
    db_start_date = datetime.strftime(df_ui['From'].min(), '%Y-%m-%d')
    db_end_date = datetime.strftime(df_ui['From'].max() + timedelta(days=6), '%Y-%m-%d')

    # adjust floodlights
    floodlights_columns = list(chain(*list_of_floodlights)) # unpack list
    floodlights_columns = list(dict.fromkeys(floodlights_columns,None).keys()) # remove dupes

    # filter metrics
    floodlights_names = df_metrics[df_metrics['sa360_col_name'].isin(floodlights_columns)]
    db_metrics = ['clicks', 'impr', 'cost']

    if len(floodlights_names['sa360_col_name'].unique()) == len(floodlights_columns):
        db_metrics = "','".join(list(floodlights_names['metric']) + db_metrics)
    else:
        db_metrics = "','".join(db_metrics)

# get data from redshift
# ========================================================================================================================
    qry_txt = get_qry_text(redshift_qa_query)
    qry_txt = qry_txt.replace('load_metrics', db_metrics) \
                     .replace('load_advertisers', db_advertisers) \
                     .replace('load_start_date', db_start_date) \
                     .replace('load_end_date', db_end_date)

    time.sleep(1)
    df_qry_raw = run_qry(qry_txt)

# map floodlights and remove nulls
# add nulls to error log
# ========================================================================================================================
    df_qry = df_qry_raw.copy()
    df_qry = df_qry.merge(floodlights_names, how='left', on=['advertiser', 'metric'])
    df_qry['sa360_col_name'] = df_qry.apply(lambda x: x['metric'].title() if x['metric'] in ['impr', 'cost', 'clicks'] else x['sa360_col_name'], axis=1)

    # null results
    df_qry_na = df_qry[df_qry['sa360_col_name'].isna()]
    df_qry_na_metrics = df_qry_na[['advertiser', 'metric']].drop_duplicates().reset_index(drop=True)

    if len(df_qry_na_metrics.merge(floodlights_names, how='inner')) > 0:
        print('advertiser and metric combination is not in df_qry, check sa360_metrics.xlsx')
        print(df_qry_na_metrics)

# set up the qa dataframe
# ========================================================================================================================
    df_qry = df_qry[-df_qry['sa360_col_name'].isna()]

    # align ui column names
    ui_col_rename = {
        'From': 'week',
        'Account': 'advertiser',
        'Campaign': 'campaign_name',
        'variable': 'sa360_col_name',
        'value': 'value'
    }

    # filter ui and database results columns
    df_ui_fltr = df_ui[ui_col_rename.keys()]
    df_ui_fltr.rename(columns=ui_col_rename, inplace=True)
    df_qry_fltr = df_qry[ui_col_rename.values()]

    # add source column
    df_qry_fltr['source'], df_ui_fltr['source'] = 'redshift', 'sa360'
    df_qa = pd.concat([df_qry_fltr, df_ui_fltr])
    df_qa['advertiser'] = df_qa['advertiser'].str.replace('Amex - Shop small', 'Amex - Shop Small')

# set up the dictionary file to save results
# ========================================================================================================================
    pvts_dict = dict()

    # details
    db_params_dict = {'last_updated':datetime.now(), 'advertisers': db_advertisers, 'start_date': db_start_date, 'end_date': db_end_date, 'sql_qry': qry_txt}
    details = pd.DataFrame.from_dict(db_params_dict, orient='index').reset_index()
    details.columns = ['variable', 'value']
    pvts_dict['details'] = details

    # create qa views
    df_v0 = df_qa.copy()
    df_v0 = df_v0.pivot_table(index=['advertiser', 'campaign_name', 'week', 'sa360_col_name'], columns=['source'], values='value', aggfunc='sum').reset_index().fillna(0)
    df_v0 = df_v0.melt(id_vars=['advertiser', 'campaign_name', 'week', 'sa360_col_name'])
    df_v0 = df_v0.sort_values(by=['advertiser', 'campaign_name', 'week', 'sa360_col_name']).reset_index(drop=True)
    df_v0.rename(columns={'sa360_col_name':'metric'}, inplace=True)

    # redshift v sa360
    df_v1 = df_qa.copy()

    df_v1 = df_v1.pivot_table(index=['advertiser', 'campaign_name', 'week', 'sa360_col_name'], columns=['source'], values='value', aggfunc='sum').reset_index()
    df_v1['%_diff'] = abs((df_v1['redshift']/df_v1['sa360'])-1)
    df_v1.rename(columns={'sa360_col_name':'metric'}, inplace=True)

    view_args = {
        'advertiser': {'index':['advertiser'],
                       'dim_cutoff': 1},

        'campaign': {'index':['advertiser', 'campaign_name'],
                     'dim_cutoff': 2},

        'week': {'index':['advertiser', 'campaign_name', 'week'],
                 'dim_cutoff': 3}
                }

    # loop to calculate difference by view args keys
    for k in view_args.keys():
        df_pvt = df_qa_week = df_qa.pivot_table(index=view_args[k]['index'], columns=['sa360_col_name', 'source'], values='value', aggfunc='sum').reset_index()

        list_of_metric_diff = []
        metrics = df_pvt.columns.levels[0][:list(df_pvt.columns.levels[0]).index(view_args[k]['index'][-1])]
        for m in metrics:
            df_temp = df_pvt.copy()
            df_temp = df_temp[m]
            df_temp.columns = [f"{m}_{c}" for c in df_temp.columns]
            df_temp[f"{m}_%_diff"] = (df_temp.iloc[:,1]/df_temp.iloc[:,0])-1
            list_of_metric_diff.append(df_temp)

        metric_diffs = pd.concat(list_of_metric_diff, axis=1)

        df_pvt_qa = pd.concat([df_pvt.iloc[:, :view_args[k]['dim_cutoff']], metric_diffs], axis=1)
        df_pvt_qa.columns = [c[0] if type(c) is tuple else c for c in df_pvt_qa.columns]

        pvts_dict[k] = df_pvt_qa

    pvts_dict['raw_data'], pvts_dict['redshift_v_sa360'] = df_v0, df_v1

    return pvts_dict

In [148]:
start = time.time()

In [149]:
print(f"runtime: {np.round(time.time() - start, 2)} seconds")

runtime: 0.51 seconds
